In [1]:
import os
import sys

if os.environ['PY_SOURCES'] not in sys.path:
    sys.path.append(os.environ['PY_SOURCES'])

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [97]:
import numpy as np
import pandas as pd
import kaggle.house_prices.helpers as helpers

combined_dataset = helpers.load_data()

In [3]:
combined_dataset.shape
combined_dataset[combined_dataset['dataSource'] == 'train'].shape
combined_dataset[combined_dataset['dataSource'] == 'test'].shape

(2919, 82)

(1460, 82)

(1459, 82)

In [81]:
colnames = helpers.get_character_colnames(combined_dataset)
colnames.remove('dataSource')
colnames.append('SalePrice')

categ_data = combined_dataset[colnames]
categ_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 45 columns):
Alley            198 non-null object
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtQual         2838 non-null object
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
Electrical       2918 non-null object
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      2918 non-null object
Exterior2nd      2918 non-null object
Fence            571 non-null object
FireplaceQu      1499 non-null object
Foundation       2919 non-null object
Functional       2917 non-null object
GarageCond       2760 non-null object
GarageFinish     2760 non-null object
GarageQual       2760 non-null object
GarageType       2762 non-null object
Heating          2919

In [84]:
colnames = helpers.get_numeric_colnames(combined_dataset)
colnames.remove('Id')

numeric_data = combined_dataset[colnames]
numeric_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 36 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
BedroomAbvGr     2919 non-null int64
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtUnfSF        2918 non-null float64
EnclosedPorch    2919 non-null int64
Fireplaces       2919 non-null int64
FullBath         2919 non-null int64
GarageArea       2918 non-null float64
GarageCars       2918 non-null float64
GarageYrBlt      2760 non-null float64
GrLivArea        2919 non-null int64
HalfBath         2919 non-null int64
KitchenAbvGr     2919 non-null int64
LotArea          2919 non-null int64
LotFrontage      2433 non-null float64
LowQualFinSF     2919 non-null int64
MasVnrArea       2896 non-null float64
MiscVal          2919 non-null int64
MoSold           2919 

In [114]:
def entropy(col):
    freqs = col.value_counts(dropna=False) / len(col)
    return -np.sum(freqs * np.log2(freqs))

In [139]:
x = (
    categ_data
    .drop('SalePrice', 1)
    .apply(entropy)
    .sort_values()
)
    
x.head(10)
x.tail()

Utilities      0.012627
PoolQC         0.038203
Street         0.038504
Condition2     0.106434
RoofMatl       0.138519
Heating        0.139775
MiscFeature    0.244488
LandSlope      0.303799
CentralAir     0.355183
Alley          0.423391
dtype: float64

BsmtFinType1    2.454724
Exterior1st     2.723429
Exterior2nd     2.800720
MSSubClass      2.920021
Neighborhood    4.212100
dtype: float64

In [138]:
x = (
    numeric_data
    .drop('SalePrice', 1)
    .apply(entropy)
    .sort_values())
    
x.head(10)
x.tail()

PoolArea        0.057677
3SsnPorch       0.158769
LowQualFinSF    0.173265
KitchenAbvGr    0.280959
BsmtHalfBath    0.344808
MiscVal         0.374903
ScreenPorch     0.991907
HalfBath        1.010771
BsmtFullBath    1.077617
FullBath        1.173785
dtype: float64

TotalBsmtSF     9.348757
BsmtUnfSF       9.390885
1stFlrSF        9.601608
GrLivArea       9.882939
LotArea        10.400602
dtype: float64

In [169]:
xxx = pd.melt(
    (
        combined_dataset[['SalePrice']]
        .dropna()
        .rename(columns={"SalePrice": "price"})
        .assign(**{'price.log':  lambda df: np.log(df.price)})
    ), 
    var_name='var', 
    value_name='value'
).assign(
    normed_value = lambda df: (
        df.groupby('var', group_keys=False)
          .apply(lambda df: (df.value - np.mean(df.value)) / np.std(df.value))
    )
)

xxx

,var,value,normed_value
0,price,208500.000000,0.347273
1,price,181500.000000,0.007288
2,price,223500.000000,0.536154
3,price,140000.000000,-0.515281
4,price,250000.000000,0.869843
5,price,143000.000000,-0.477505
6,price,307000.000000,1.587588
7,price,200000.000000,0.240241
8,price,129900.000000,-0.642461
9,price,118000.000000,-0.792306
